# Part 1

In [13]:
# Stolen from day 16 and modified here:
import functools
import operator

class CpuHalt(Exception):
    pass

class Cpu:
    def __init__(self, registers):
        self._registers = registers
        self._ip = 0
        self._ip_register = None
        self._program = None

    @property
    def registers(self):
        return tuple(self._registers)

    def program(self, text):
        ''' Parse program text and insert it into this CPU. '''
        lines = text.strip().split('\n')
        program = list()
        ip_register = None
        for line in lines:
            if line.startswith('#ip'):
                _, reg = line.split()
                self._ip_register = int(reg)
            else: 
                insn, a, b, c = line.split()
                program.append((insn, int(a), int(b), int(c)))
        self._program = program

    def step(self, debug=False):
        ''' Run next instruction and update IP. '''
        try:
            instruction = self._program[self._ip]
        except IndexError:
            raise CpuHalt() 
        self._registers[self._ip_register] = self._ip
        if debug:
            before = self.registers
        op = getattr(self, instruction[0])
        op(*instruction[1:4])
        if debug:
            after = self.registers
            print('ip={:<3d} {} {} {}'.format(self._ip, before, instruction, after))
        self._ip = self._registers[self._ip_register]
        self._ip += 1
    
    def run(self, n=10_000_000, debug=False):
        ''' Run until the CPU halts, i.e. the IP points to a non-existent
        instruction. '''
        try:
            for _ in range(n):
                self.step(debug)
            print('Finished running {} steps.'.format(n))
        except CpuHalt:
            print('CPU halted. Registers={}'.format(self.registers))
    
    def _do_op_imm(self, op, a, b, c):
        self._registers[c] = op(self._registers[a], b)
        
    def _do_op_reg(self, op, a, b, c):
        self._registers[c] = op(self._registers[a], self._registers[b])

    addi = functools.partialmethod(_do_op_imm, operator.add)
    addr = functools.partialmethod(_do_op_reg, operator.add)
    muli = functools.partialmethod(_do_op_imm, operator.mul)
    mulr = functools.partialmethod(_do_op_reg, operator.mul)
    bani = functools.partialmethod(_do_op_imm, operator.and_)
    banr = functools.partialmethod(_do_op_reg, operator.and_)
    bori = functools.partialmethod(_do_op_imm, operator.or_)
    borr = functools.partialmethod(_do_op_reg, operator.or_)

    def _do_cmp_reg_imm(self, cmp, a, b, c):
        self._registers[c] = int(cmp(self._registers[a], b))
        
    def _do_cmp_imm_reg(self, cmp, a, b, c):
        self._registers[c] = int(cmp(a, self._registers[b]))

    def _do_cmp_reg_reg(self, cmp, a, b, c):
        self._registers[c] = int(cmp(self._registers[a], self._registers[b]))

    gtir = functools.partialmethod(_do_cmp_imm_reg, operator.gt)
    gtri = functools.partialmethod(_do_cmp_reg_imm, operator.gt)
    gtrr = functools.partialmethod(_do_cmp_reg_reg, operator.gt)
    eqir = functools.partialmethod(_do_cmp_imm_reg, operator.eq)
    eqri = functools.partialmethod(_do_cmp_reg_imm, operator.eq)
    eqrr = functools.partialmethod(_do_cmp_reg_reg, operator.eq)

    def seti(self, a, b, c):    
        self._registers[c] = a
        
    def setr(self, a, b, c):
        self._registers[c] = self._registers[a]

In [17]:
test_text = '''#ip 0
seti 5 0 1
seti 6 0 2
addi 0 1 0
addr 1 2 3
setr 1 0 0
seti 8 0 4
seti 9 0 5
'''
test_cpu = Cpu(registers=[0,0,0,0,0,0])
test_cpu.program(test_text)
print(test_cpu.registers)

(0, 0, 0, 0, 0, 0)


In [18]:
test_cpu.run(debug=True)

ip=0   (0, 0, 0, 0, 0, 0) ('seti', 5, 0, 1) (0, 5, 0, 0, 0, 0)
ip=1   (1, 5, 0, 0, 0, 0) ('seti', 6, 0, 2) (1, 5, 6, 0, 0, 0)
ip=2   (2, 5, 6, 0, 0, 0) ('addi', 0, 1, 0) (3, 5, 6, 0, 0, 0)
ip=4   (4, 5, 6, 0, 0, 0) ('setr', 1, 0, 0) (5, 5, 6, 0, 0, 0)
ip=6   (6, 5, 6, 0, 0, 0) ('seti', 9, 0, 5) (6, 5, 6, 0, 0, 9)
CPU halted. Registers=(6, 5, 6, 0, 0, 9)


In [20]:
cpu = Cpu(registers=[0,0,0,0,0,0])
with open('input.txt') as input_:
    cpu.program(input_.read())

In [21]:
%%time
cpu.run()

CPU halted. Registers=(930, 930, 1, 930, 256, 929)
CPU times: user 14.3 s, sys: 3.53 ms, total: 14.3 s
Wall time: 14.3 s


# Part 2

In [22]:
cpu = Cpu(registers=[1,0,0,0,0,0])
with open('input.txt') as input_:
    cpu.program(input_.read())

In [23]:
print(cpu.registers)

(1, 0, 0, 0, 0, 0)


In [24]:
%%time
# This cell doesn't work. See below.
# cpu.run()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [25]:
%%time
# The CPU can run hundreds of millions of instructions without halting, which makes 
# me think that this problem involves some huge loop and I'm expected to predict the
# answer rather than running the CPU until it halts. So I think I will run it for
# a while, then break and single-step through to see what's happening.
cpu.run()

Finished running 10000000 steps.
CPU times: user 20.4 s, sys: 3.96 ms, total: 20.4 s
Wall time: 20.4 s


In [26]:
# Now the CPU has run for 10M steps without halting. Lets run the next 50 and see
# if there is a pattern.
cpu.run(n=50, debug=True)

ip=6   (0, 1249998, 0, 1, 6, 10551329) ('addi', 4, 1, 4) (0, 1249998, 0, 1, 7, 10551329)
ip=8   (0, 1249998, 0, 1, 8, 10551329) ('addi', 1, 1, 1) (0, 1249999, 0, 1, 8, 10551329)
ip=9   (0, 1249999, 0, 1, 9, 10551329) ('gtrr', 1, 5, 2) (0, 1249999, 0, 1, 9, 10551329)
ip=10  (0, 1249999, 0, 1, 10, 10551329) ('addr', 4, 2, 4) (0, 1249999, 0, 1, 10, 10551329)
ip=11  (0, 1249999, 0, 1, 11, 10551329) ('seti', 2, 9, 4) (0, 1249999, 0, 1, 2, 10551329)
ip=3   (0, 1249999, 0, 1, 3, 10551329) ('mulr', 3, 1, 2) (0, 1249999, 1249999, 1, 3, 10551329)
ip=4   (0, 1249999, 1249999, 1, 4, 10551329) ('eqrr', 2, 5, 2) (0, 1249999, 0, 1, 4, 10551329)
ip=5   (0, 1249999, 0, 1, 5, 10551329) ('addr', 2, 4, 4) (0, 1249999, 0, 1, 5, 10551329)
ip=6   (0, 1249999, 0, 1, 6, 10551329) ('addi', 4, 1, 4) (0, 1249999, 0, 1, 7, 10551329)
ip=8   (0, 1249999, 0, 1, 8, 10551329) ('addi', 1, 1, 1) (0, 1250000, 0, 1, 8, 10551329)
ip=9   (0, 1250000, 0, 1, 9, 10551329) ('gtrr', 1, 5, 2) (0, 1250000, 0, 1, 9, 10551329)
ip=10 

There is clearly a loop: 3,4,5,6,8,9,10,11 (skips 7!).

Here are the contents of the loop cleaned up. I've added registers names to make the
pseudocode below clearer:

            0  1        2        3  4   5
            A  B        C        D  IP  F
           --------------------------------------
    ip=3   (0, 1249999, 0,       1, 3,  10551329)  ('mulr', 3, 1, 2)
    ip=4   (0, 1249999, 1249999, 1, 4,  10551329)  ('eqrr', 2, 5, 2)  
    ip=5   (0, 1249999, 0,       1, 5,  10551329)  ('addr', 2, 4, 4)  
    ip=6   (0, 1249999, 0,       1, 6,  10551329)  ('addi', 4, 1, 4)  
    ip=8   (0, 1249999, 0,       1, 8,  10551329)  ('addi', 1, 1, 1)  
    ip=9   (0, 1250000, 0,       1, 9,  10551329)  ('gtrr', 1, 5, 2)
    ip=10  (0, 1250000, 0,       1, 10, 10551329)  ('addr', 4, 2, 4)
    ip=11  (0, 1250000, 0,       1, 11, 10551329)  ('seti', 2, 9, 4)

    ip=3   (0, 1250000, 0,       1, 3,  10551329)  ('mulr', 3, 1, 2)
    ip=4   (0, 1250000, 1250000, 1, 4,  10551329)  ('eqrr', 2, 5, 2)
    ip=5   (0, 1250000, 0,       1, 5,  10551329)  ('addr', 2, 4, 4)
    ip=6   (0, 1250000, 0,       1, 6,  10551329)  ('addi', 4, 1, 4)
    ip=8   (0, 1250000, 0,       1, 8,  10551329)  ('addi', 1, 1, 1)
    ip=9   (0, 1250001, 0,       1, 9,  10551329)  ('gtrr', 1, 5, 2)
    ip=10  (0, 1250001, 0,       1, 10, 10551329)  ('addr', 4, 2, 4)
    ip=11  (0, 1250001, 0,       1, 11, 10551329)  ('seti', 2, 9, 4)

It's worth nothing that register B appears to be incrementing, while the other
registers stay constant or step through a repeating sequence. Also, we ran 10M
steps and the B register is 1.25M, indicating it increments once every 8 steps,
which is exactly how long this loop is. This suggests that the CPU has spent
almost all of its time inside this loop.

Here's pseudocode for the program.

    00  IP += 16 // Jump to init_f
                 // Notice that this would have no effect in part 1, because
                 // this was not the IP register in part 1.
                 
    main:
    
    01  D = 1    // Initialize B and D to 1
    02  B = 1
    
    loop1:
    
    03  C = D * B                // D is always 1, so this is copy B → C
    04  C = (C==F) ? 1 : 0
    05  IP += C                  // If F==D*B, then jump to 7
    06  IP += 1                  // Jump to 8
    07  A += D                   // A += D
    08  B += 1
    09  C = (B > F) ? 1 : 0
    10  IP += C                  // If B>F, then jump to 12
    11  IP = 2                   // Goto loop1
    
    loop2:
    
    12 D += 1      // D++
    13 C = (D > F) ? 1 : 0
    14 IP += C     // If D > F, Jump to 16
    15 IP = 1      // Jump to 2
    16 IP *= IP    // Halt: IP = 16*16 = 256
    
    init_f:
    
    17  F += 2
    18  F *= F     // F = F^2
    19  F *= IP    // F *= 19
    20  F *= 11
    21  C += 4
    22  C *= IP    // C *= 22
    23  C += 5
    24  F += C
    25  IP += A    // A is initialized to 1 before the program starts
                   // So this jumps to 27
    26  IP = 0     // Jump to main (I don't think this ever gets executed)
    27  C = IP     // C = 27
    28  C *= IP    // C *= 28
    29  C += IP    // C += 29
    30  C *= IP    // C *= 30
    31  C *= 14    // C *= 14
    32  C *= IP    // C *= 32
    33  F += C
    34  A = 0
    35  IP = 0     // Jump to main

It looks like the code has three lines of initialization (lines 0-2), a loop (lines 3-11), and a
function to initialize f (lines 17-35). To confirm that my pseudo code is correct, we can trace the initialization of F and confirm that it matches the value shown in the steps above.

In [27]:
F = 0
F += 2
F *= F
F *= 19
F *= 11
print(F)

836


In [28]:
C = 0
C += 4
C *= 22
C += 5
print(C)

93


In [29]:
F += C
print(F)

929


In [30]:
C = 27
C *= 28
C += 29
C *= 30
C *= 14
C *= 32
print(C)

10550400


In [31]:
F += C
print(F)

10551329


The code appears to contain nested loops. The inner loop1 (lines 3-11) increments B until it is greater than F. Then outer loop2 (lines 12-16) increments D, sets B back to 1, and goes back into loop1. So these loops appear to execute F^2 times, i.e. 10,551,329 * 10,551,329 = 111,330,543,666,241, which is completely infeasible to execute.

The value of interest is A, and A is only updated in one place (line 7), which is only executed
when F == D * B. When that is true, then A += D. I think this is finding factors of F in the most ridiculously slow way possible, and storing the sum in A.

I used [this website](https://www.numberempire.com/numberfactorizer.php) to factorize 10,551,329 and got 137 and 77,017, both of which are prime. So A should be the sum of these two numbers.

In [32]:
# Wrong, too low!
137 + 77017

77154

In [33]:
# Whoops, I need to add in the trivial factors!
1 + 137 + 77017 + 10551329

10628484